# GPU 사용 설정
- 런타임 - 런타임 유형 변경 -  GPU 선택

In [ ]:
# GPU 사용 가능 여부 확인
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())
print("GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "사용 불가")

# 구글 드라이브 DATA 확인
- 구글 드라이브에 업로드된 데이터 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# 데이터 목록 확인
!ls /content/drive/My\ Drive/Colab\ Notebooks/data

In [ ]:
import os

# 작업 디렉터리를 원하는 경로로 변경
current_dir = "/content/drive/My Drive/Colab Notebooks/"
os.chdir(current_dir)

# 변경된 현재 작업 디렉터리 확인
print("Current working directory:", os.getcwd())

# Transformers 버전 확인

In [ ]:
import transformers
transformers.__version__

# RDkit 설치

In [ ]:
!pip install rdkit

# 필요 모듈 로딩

In [ ]:
from transformers import (
    EncoderDecoderModel,
    Trainer, TrainingArguments,
    BertConfig, EncoderDecoderConfig, DataCollatorForSeq2Seq
)

from datautil import (
    get_train_val_dataset, tokenize_function, train_tokenizer)

# Tokenizer 학습

In [ ]:
# Tokenizer 학습
tokenizer = train_tokenizer([
    "./data/src-train.txt",
    "./data/tgt-train.txt"
])

In [ ]:
# 토크나이저 인코딩 확인
tokenizer.encode("C C ( = O ) O c 1 c c c c c 1 C ( = O ) O")

# 학습/검증 데이터 불러오기

In [ ]:
# Dataset 구성
raw_datasets = get_train_val_dataset()
tokenized_datasets = raw_datasets.map(
    lambda x: tokenize_function(x, tokenizer), batched=True)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_datasets['train']['input_ids'][0])

# 모델 정의

In [ ]:
# Config 정의 및 모델 생성
config = EncoderDecoderConfig.from_encoder_decoder_configs(
    BertConfig(
        vocab_size=tokenizer.vocab_size,
        bos_token_id=tokenizer.convert_tokens_to_ids("[BOS]"),
        eos_token_id=tokenizer.convert_tokens_to_ids("[EOS]"),
        pad_token_id=tokenizer.convert_tokens_to_ids("[PAD]"),
        hidden_size=256,
        num_hidden_layers=4,
        num_attention_heads=4,
        intermediate_size=256,
    ),
    BertConfig(
        vocab_size=tokenizer.vocab_size,
        bos_token_id=tokenizer.convert_tokens_to_ids("[BOS]"),
        eos_token_id=tokenizer.convert_tokens_to_ids("[EOS]"),
        pad_token_id=tokenizer.convert_tokens_to_ids("[PAD]"),
        hidden_size=256,
        num_hidden_layers=4,
        num_attention_heads=4,
        intermediate_size=256,
    )
)
model = EncoderDecoderModel(config=config)
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

# 학습 설정 및 학습 진행

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # no parallelism

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./checkpoints",
    learning_rate=5e-4, #5e-4
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1, # just one epoch!
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",        # 평가 주기 설정 필수
    load_best_model_at_end=True,        # 가장 좋은 모델 로드
    metric_for_best_model="eval_loss",  # 기준 metric 설정
    greater_is_better=False,             # 낮은 loss가 더 좋으므로 False
    report_to="tensorboard",  # TensorBoard에 로그 기록
    logging_dir="./logs",
    logging_strategy="epoch"
)

In [ ]:
# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

# 학습 시작
trainer.train()